<h1>Introduction</h1>

The goal of the exercise is twofold:
- To provide an overview of the state of the art implementation of a convolutional neural network (in our case ResNet 50) including all of the elements discussed in the previous lectures and labs (data input pipeline considerations, NCCL implementation, etc.)
- To Illustrate how this example can be used as a template for further neural network development. We will implement a new neural network, “Alexnet OWT” as described in https://arxiv.org/abs/1404.5997.


<h1>Horovod</h1>

<h2>Rationale</h2>
Historically, for all the reasons discussed in this class, training large scale jobs was technically challenging. One of the key factors affecting our ability to scale was communication which had a non-trivial impact on the overall training speed as illustrated below: 

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_HorovodRationale.png"/>

Especially TensorFlow, which relied heavily on the presence of parameter servers when used with Synchronous SGD was experiencing substantial communication bottlenecks. As you can see in the diagrams below, as you increase the number of processes participating in training, and therefore communication, communication requirement becomes higher. This can be partially addressed by increasing the number of parameter servers, what also adds substantial complexity to the overall system.

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_HorovodParameterService.png"/>

The development of NVLINK and adaptation of HPC like communication patterns and especially All Reduce algorithm was one of the key cornerstones that allowed to overcome this challenge. It not only substantially reduces the overall amount of communication but also removes the need for the parameter server which in certain scenarios was a communication bottleneck. The diagram below illustrates how gradient exchange is implemented using the All reduce algorithm.

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_HorovodAllReduce.png"/>

Hiding this communication and implementation complexity is the core goal of Horovod. In this lab we will see how to use Horovod for distributed training. Throughout this exercise it will become very clear that we are not participating in the implementation of the gradient exchange logic.

<h2>Transforming single GPU code to Horovod multi GPU implementation</h2>

As discussed in the section above, one of the key goals of Horovod was to simplify the complexity of writing efficient distributed software. Therefore, migrating your single GPU model to a large multi GPU or even multi node setup is straightforward. The key software changes that need to be introduced in your code are as follows:

<ul>
    <li><b>hvd.init()</b> initializes Horovod.</li>
    <li><b>config.gpu_options.visible_device_list = str(hvd.local_rank())</b> assigns a GPU to each of the TensorFlow processes (this code needs to be slightly adjusted if you want to mix the data and model parallel implementation).</li>
    <li><b>opt=hvd.DistributedOptimizer(opt)</b> wraps any regular TensorFlow optimizer with Horovod optimizer which takes care of averaging gradients using ring-all reduce.</li>
    <li><b>hvd.BroadcastGlobalVariablesHook(0)</b> broadcasts variables from the first process to all other processes. This can be used together with the MonitoredTrainingSession or if it is not used (like in this example) it can be called directly via the hvd.broadcast_global_variables(0) operations instead.</li>
</ul>

With those software components in mind let us focus on code review of our distributed algorithm which will be accompanied by several simple code exercises.

<h2>Our example</h2>

Before we dive into the implementation detail let us execute the code we will be reviewing in this exercise.

In [1]:
! mpiexec --allow-run-as-root -np 2 python3 nvcnn_hvd_simplified.py \
                      --model=resnet50 \
                      --data_dir=/dli/data/tfdata352p90per \
                      --batch_size=48 

--------------------------------------------------------------------------
[[54403,1],0]: A high-performance Open MPI point-to-point messaging module
was unable to find any relevant network interfaces:

Module: OpenFabrics (openib)
  Host: 37e56af97b35

Another transport will be used instead, although this may result in
lower performance.

NOTE: You can disable this warning by setting the MCA parameter
btl_base_warn_component_unused to 0.
--------------------------------------------------------------------------
Cmd line args:
  --model=resnet50
  --data_dir=/dli/data/tfdata352p90per
  --batch_size=48
Num ranks:   2
Num images:  1000
Model:       resnet50
Batch size:  48 per device
             96 total
Data format: NCHW
Data type:   fp32
Building training graph
[37e56af97b35:00094] 1 more process has sent help message help-mpi-btl-base.txt / btl:no-nics
[37e56af97b35:00094] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages
Creating session
2019-03-11 0

As you might have noticed, we are not executing the code directly. Instead we are relying on <a href="https://en.wikipedia.org/wiki/Message_Passing_Interface">mpiexec</a> to distribute the execution of our program (<a href="https://en.wikipedia.org/wiki/Message_Passing_Interface">mpiexec</a> is a part of Message Passing Interface (MPI) which is a standard for writing portable parallel programmes. Horovod uses MPI as a mechanism to distribute its execution.). The syntax is straightforward. We pass the name of the program we want to execute, in our case <b>"python nvcnn_hvd_simplified.py"</b> followed by several program specific parameters. We also define number of GPUs we want to use.
<br/><br/>
To execute the code on more than one machine we would also provide a list of hosts which will be participating in the computation and increase the number of GPUs used.
<br/>
<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_HorovodDist.png" width=300/>

<h2>Code structure</h2>

Our code is composed of four major blocks:
<ul>
    <li><a href="../../../../edit/tasks/task2/task/GPUNetworkBuilder.py">GPUNetworkBuilder.py</a> which contains the key building blocks for creation of Convolutional Neural Networks.</li>
    <li><a href="../../../../edit/tasks/task2/task/ImagePreprocessor.py">ImagePreprocessor.py</a> which defines a set of basic primitives for data loading, decoding and augmentation.</li>
    <li><a href="../../../../edit/tasks/task2/task/FeedForwardTrainer.py">FeedForwardTrainer.py</a> which defines the end to end training pipeline bringing together the key code components. This includes taking advantage of the ImagePreprocessor routines to build a multithreaded and asynchronous input pipeline but also definition of the training step and the optimization regime.</li>
    <li>Model definition. In our case we will implement two models, hence two files, <a href="../../../../edit/tasks/task2/task/AlexNet.py">AlexNet.py</a> and <a href="../../../../edit/tasks/task2/task/ResNet.py">ResNet.py</a></li>
</ul>

The entire training functionality (taking advantage of the above mentioned files) is defined in <a href="../../../../edit/tasks/task2/task/nvcnn_hvd_simplified.py">nvcnn_hvd_simplified.py</a>.

<h2>GPUNetworkBuilder</h2>

<a href="../../../../edit/tasks/task2/task/GPUNetworkBuilder.py">GPUNetworkBuilder.py</a> is a very simple class that implements key building blocks of modern Convolutional Neural Networks. It implements operations such as pooling:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_PoolingLayer.png" width=600/>

or a wide range of activation functions:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_ActivateLayer.png" width=600/>

In order to use the functionality defined in the class, we need to create an instance of the class:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_GPUNetworkBuilder.png"/>

and then use it directly to define our model.


<h2>Model definition files</h2>

The <a href="../../../../edit/tasks/task2/task/GPUNetworkBuilder.py">GPUNetworkBuilder.py</a>  is then used by the  <a href="../../../../edit/tasks/task2/task/AlexNet.py">AlexNet.py</a> and <a href="../../../../edit/tasks/task2/task/ResNet.py">ResNet.py</a> to define the shape of the models we will be training. For example it is used in <a href="../../../../edit/tasks/task2/task/AlexNet.py">AlexNet.py</a> to define the shape of the AlexNet neural network:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_AlexNetInference.png" width=500/>

<h2>Adding a new model</h2>

In this part of the exercise we will extend the capability of our script by introducing a new model, AlexNet. As you have already seen during this exercise our source code is very modular so adding a new model is limited to:
<ul>
    <li>Defining our model through the inference function</li>
    <li>Extending our command line interface to allow the user to select AlexNet as a model and pass model specific hyperparameters to the trainer.
</ul>

Let us look at those steps one at a time.

<h3>Defining the AlexNet model</h3>

Since we have collected all of the key building blocks of CNN networks in previously discussed <a href="../../../../edit/tasks/task2/task/GPUNetworkBuilder.py">GPUNetworkBuilder.py</a> creation of a new model (especially as simple as AlexNet) is straightforward. Because of the time constraints of our class we will not be building the model from scratch, instead we will integrate the model as defined in <a href="../../../../edit/tasks/task2/task/AlexNet.py">AlexNet.py</a> to our program. The file <a href="../../../../edit/tasks/task2/task/AlexNet.py">AlexNet.py</a> contains a single function that takes two parameters:
<ul>
    <li>Our network builder object</li>
    <li>Input layer that we will use to deliver raw image data to our network</li>
</ul>

The function returns a computational graph defining the AlexNet model which we can then pass to our distributed optimization logic.

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_AlexNetInference.png" width=500/>

<h3>Integration</h3>

Let us now try to integrate the model defined above with the rest of our code. We will be working with the <b><a href="../../../../edit/tasks/task2/task/nvcnn_hvd_noAlexNet.py">nvcnn_hvd_noAlexNet.py</a></b> file. If at any point you would like to inspect a working solution do refer to the <a href="../../../../edit/tasks/task2/task/nvcnn_hvd_simplified.py">nvcnn_hvd_simplified.py</a> file.
<br/><br/>
First step is to import our model. Please identify a section in <a href="../../../../edit/tasks/task2/task/nvcnn_hvd_noAlexNet.py">nvcnn_hvd_noAlexNet.py</a> where we are importing the ResNet model and follow the same structure to import the AlexNet model. E.g.:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_AlexNetImport.png" width=600/>
<br/><br/>
Secondly we need to:
<ul>
    <li>Identify the section which is responsible for handling user input and define a conditional statement which will be executed when the user selects to execute the AlexNet network</li>
    <li>In that section we need to set AlexNet as a model we are going to optimize</li>
    <li>Finally we need to define model specific parameters. In this case the image dimensions and the learning rate.</li>
</ul>

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_AlexNetControll.png" width=600/>

<br/><br/>
<b>Remember to save your changes!</b>
<br/><br/>

Once the changes are saved executing the code is straightforward:

In [2]:
! mpiexec --allow-run-as-root -np 2 python3 nvcnn_hvd_noAlexNet.py \
                      --model=alexnet \
                      --data_dir=/dli/data/tfdata352p90per \
                      --batch_size=48 

--------------------------------------------------------------------------
[[53452,1],0]: A high-performance Open MPI point-to-point messaging module
was unable to find any relevant network interfaces:

Module: OpenFabrics (openib)
  Host: 37e56af97b35

Another transport will be used instead, although this may result in
lower performance.

NOTE: You can disable this warning by setting the MCA parameter
btl_base_warn_component_unused to 0.
--------------------------------------------------------------------------
Cmd line args:
  --model=alexnet
  --data_dir=/dli/data/tfdata352p90per
  --batch_size=48
Num ranks:   2
Num images:  1000
Model:       alexnet
Batch size:  48 per device
             96 total
Data format: NCHW
Data type:   fp32
Traceback (most recent call last):
  File "nvcnn_hvd_noAlexNet.py", line 302, in <module>
    main()
  File "nvcnn_hvd_noAlexNet.py", line 142, in main
    raise ValueError("Invalid model type: %s" % model_name)
ValueError: Invalid model type: alexnet
T

<b>If you have any problems with the code above feel free to execute the working version of the code (which you can inspect <a href="../../../../edit/tasks/task2/task/nvcnn_hvd_simplified.py">here</a>).</b>

In [ ]:
! mpiexec --allow-run-as-root -np 2 python3 nvcnn_hvd_simplified.py \
                      --model=alexnet \
                      --data_dir=/dli/data/tfdata352p90per \
                      --batch_size=48 

<h2>Input pipeline</h2>

The <a href="../../../../edit/tasks/task2/task/ImagePreprocessor.py">ImagePreprocessor.py</a> implements a collection of routines that will be further used to implement a multithreaded and asynchronous input pipeline. It implements operations such as input decoding:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_DecodeJPEG.png" width=400/>

Wide range of data augmentation operations:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_Augmentation.png" width=500/>

As well as a multithreaded data loading process:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_ParallelLoad.png" width=400/>

To get a better understanding of the code structure let us change the behavior of the data input pipeline by:
<ul>
    <li>Changing the JPEG decoder used. This will help you identify the portions of code responsible for key loading stages so that you learn how to replace the data type or data structure in your day to day deep learning projects</li>
    <li>Extending the data augmentation pipeline. Again this will teach you how to systematically approach the augmentation process allowing you to apply this skill in project.</li> 
</ul>

<h3>Data loading logic</h3>


The data loading logic is composed of three parts:
<ul>
    <li>Logic responsible for loading the files from the file system. Please note that we are not using python or OpenCV specific data manipulation logic. Instead we are using highly parallel data manipulation logic that comes together with TensorFLow. In our case we will be spawning 64 threads to support the process. <img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_ParallelLoad.png" width=400/></li>
    <li>Logic de-serializing our data. As you will remember from the lecture we are not working on the image data directly. Instead in this case we have stored them in a TensorFlow specific data storage format called TFRecord. The function listed below is responsible for reserializing this data representation and extracting the raw image data. <img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_DeserializeJPEG.png" width=400/></li>
    <li>In our case the data is stored in JPEG format (this is not always possible though as lossy compression introduces changes to the data unavoidably removing information from our dataset). The function below is responsible for its efficient decode. <img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_JPEGDecode.png" width=400/></li>
</ul>


Let us find the above logic responsible for JPEG decoding in <a href="../../../../edit/tasks/task2/task/ImagePreprocessor.py">ImagePreprocessor.py</a> and change the decoder used from "INTEGER_FAST" to "INTEGER_ACCURATE". Once you have made the suggested changes remember to save the file and execute the training on AlexNet network again to see the implications of the change made.

In [3]:
! mpiexec --allow-run-as-root -np 2 python3 nvcnn_hvd_simplified.py \
                      --model=alexnet \
                      --data_dir=/dli/data/tfdata352p90per \
                      --batch_size=48

--------------------------------------------------------------------------
[[53362,1],1]: A high-performance Open MPI point-to-point messaging module
was unable to find any relevant network interfaces:

Module: OpenFabrics (openib)
  Host: 37e56af97b35

Another transport will be used instead, although this may result in
lower performance.

NOTE: You can disable this warning by setting the MCA parameter
btl_base_warn_component_unused to 0.
--------------------------------------------------------------------------
Cmd line args:
  --model=alexnet
  --data_dir=/dli/data/tfdata352p90per
  --batch_size=48
Num ranks:   2
Num images:  1000
Model:       alexnet
Batch size:  48 per device
             96 total
Data format: NCHW
Data type:   fp32
Building training graph
Creating session
[37e56af97b35:01199] 1 more process has sent help message help-mpi-btl-base.txt / btl:no-nics
[37e56af97b35:01199] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages
2019-03-11 07:

Since we have implemented a more costly JPEG decoding algorithm and our neural network is CPU compute bound we have observed a degradation in overall training performance.

<h3>Augmentation logic</h3>

Our example implements the data decoding and augmentation logic in the following function:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_PreprocessJPEG.png" width=500/>

As you can see augmentation is composed of multiple image transformations. For example, the code snippet below illustrates logic used for color distortion:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_Augmentation.png" width=500/>


Let us introduce an additional augmentation step in <a href="../../../../edit/tasks/task2/task/ImagePreprocessor.py">ImagePreprocessor.py</a>.

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_RandomFlip.png" width=400/>

Once you have introduced the changes and saved the file execute the below script again. Did you observe a further decrease in training performance?

In [ ]:
! mpiexec --allow-run-as-root -np 2 python3 nvcnn_hvd_noAlexNet.py \
                      --model=alexnet \
                      --data_dir=/dli/data/tfdata352p90per \
                      --batch_size=48

<h2>Training logic</h2>

The <a href="../../../../edit/tasks/task2/task/FeedForwardTrainer.py">FeedForwardTrainer.py</a> implements the core of our training logic. It implements the training step function which we will loop through in our main code:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_TrainingStep.png" width = 300/>

Now that we have all our building blocks defined the training step function becomes straightforward. We start by assembling our input pipeline:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_InputPipeline.png" width = 400/>

We define the loss function by using the model selected (so in our case either AlexNet or ResNet50 inference function):

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_LossFunction.png" width = 400/>

We define the optimizer that we want to use and pass it to Horovod so that it can be distributed across the cluster:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_DistributedOptimiser.png" width = 400/>

We also implement the logic required to synchronize the gradients across all GPUs:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_syncHorovod.png" width = 400/>

We bring everything together and build our computational graph:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_TrainingGraph.png" width=400/>

And execute it in a training loop:

<img src="https://developer.download.nvidia.com/training/images/C-MG-01-V1_task2_img_TrainingLoop.png" width=400/>

In the code above, we have demonstrated the synchronization logic, but we have not seen it being called anywhere in the code. Please find the call to the sync() function depicted above in <a href="../../../../edit/tasks/task2/task/nvcnn_hvd_simplified.py">nvcnn_hvd_simplified.py</a>. Why is it outside of the training loop? How does this approach further simplify the implementation? Discuss it with the trainer.

<h2>Summary</h2>

As discussed at the beginning of class, Horovod allows us to ignore all the implementation detail related to distribution of our model including engineering related to model assignment to the GPU and communication. To distribute the code, we only had to implement the model and use Horovod specific optimizer and synchronization routines. All the remaining code is identical to a single GPU implementation.